In [1]:
import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal
from scipy import stats
import pywt

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from btgym.a3c import Launcher, BaseLSTMPolicy

import tensorflow as tf
import tensorflow.contrib.rnn as rnn

In [41]:
class LSTMPolicy1D(BaseLSTMPolicy):
    """
    +++++
    """
    def __init__(self, ob_space, ac_space, num_filters=32, filter_size=3, stride=2,
                 lstm_class=rnn.BasicLSTMCell, lstm_layers=(256,)):
        
        self.x = x = tf.placeholder(tf.float32, [None] + list(ob_space), name='x_in_pl')
        
        #print('GOT HERE 0')
        #print ('x:', x.shape)
    
        # Conv layers:
        for i in range(4):
            x = tf.nn.elu(self.conv1d(x, num_filters, "l{}".format(i + 1), filter_size, stride))
        
        #print('GOT HERE 1')
        #print('x_convolved:', x)
        
        # Run LSTM along rollout time dimension and evrything else:
        super(LSTMPolicy1D, self).__init__(x, ob_space, ac_space, lstm_class, lstm_layers)
        
        
    def conv1d(self, x, num_filters, name, filter_size=3, stride=2, pad="SAME", dtype=tf.float32,
               collections=None):
        with tf.variable_scope(name):
            stride_shape =  stride
            
            #print('stride_shape:',stride_shape)
            
            filter_shape = [filter_size, int(x.get_shape()[-1]), num_filters]
            
            #print('filter_shape:', filter_shape)
            
            # there are "num input feature maps * filter height * filter width"
            # inputs to each hidden unit
            fan_in = np.prod(filter_shape[:2])
            
            # each unit in the lower layer receives a gradient from:
            # "num output feature maps * filter height * filter width" /
            #   pooling size
            fan_out = np.prod(filter_shape[:1]) * num_filters

            # initialize weights with random weights
            w_bound = np.sqrt(6. / (fan_in + fan_out))

            w = tf.get_variable("W", filter_shape, dtype, tf.random_uniform_initializer(-w_bound, w_bound),
                                collections=collections)
            b = tf.get_variable("b", [1, 1, num_filters], initializer=tf.constant_initializer(0.0),
                                collections=collections)
            return tf.nn.conv1d(x, w, stride_shape, pad) + b
   

In [6]:
tf.reset_default_graph()
#pi = LSTMPolicy1D((10,2),(3))

In [63]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy,self).__init__(**kwargs)
        self.order_penalty = 1
        self.trade_just_closed = False
        self.trade_result = None
        
    def notify_trade(self, trade):    
        if trade.isclosed:
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
        
    def get_state(self):
        T = 1e4 
      
        x = np.stack(
            [self.raw_state[:,1], - self.raw_state[:,2]],  # invert `Low` values
            axis=-1
        )
        
        d_x = np.gradient(x, axis=0) * T
        
        #x[:,0] -= self.raw_state[0,0] 
        #x[:,1] += self.raw_state[0,0]
        #d_x =  x * 1e2
        

        self.state['raw_state'] = self.raw_state
        self.state['model_input'] = d_x
        
        #print('raw_state:', self.state['raw_state'].shape)
        #print('model_input:', self.state['model_input'] .shape)
        
        return self.state
    
    def get_reward(self):
        """
        Defines reward as function of last closed trade result with penalty for erronious order placement.
        """
        # Low-value term:
        r = (self.broker.get_value() / self.env.broker.startingcash - 1) * 10
        
        # Result (main-value):
        if self.trade_just_closed:
            r += self.trade_result
            self.trade_just_closed = False
            
        # Penalty for failed order:
        if self.order_failed:
            r -= self.order_penalty
            self.order_failed = False

        return r / 20  # sinse reward is just quick-example, denominator is here to keep gradients sane
    
# Set backtesting engine parameters:

time_embed_dim = 90

state_shape = {
    'raw_state': spaces.Box(low=-100, high=100, shape=(time_embed_dim, 4)),
    'model_input': spaces.Box(low=-100, high=100, shape=(time_embed_dim, 2))
}

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    MyStrategy,
    state_shape=state_shape,
    #portfolio_actions=('close_hold', 'buy_hold', 'sell_hold'),
    portfolio_actions=('hold', 'buy', 'sell'),
    drawdown_call=5, # max to loose, in percent of initial cash
    target_call=10,  # max to win, same
    skip_frame=20,
)

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)

MyCerebro.addanalyzer(bt.analyzers.DrawDown)

MyDataset = BTgymDataset(
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
    filename='../examples/data/DAT_ASCII_EURUSD_M1_201703.csv',
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_201704.csv',
    #filename='../examples/data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=55,
    start_00=False,
    time_gap_hours=8,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=False,
    render_ylabel='H/L gradients',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)

# Set tensorflow distributed cluster and a3c configuration:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_test',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    policy_class=LSTMPolicy1D,
    policy_config={'lstm_layers': (256,)},
    rollout_length=20,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=20,
    episode_summary_freq=1,
    env_render_freq=10,
    verbose=2
    
)

[2017-09-08 17:12:59,746] Launcher ready.


In [64]:
launcher.run()

[2017-09-08 17:13:00,347] parameters_server started.
[2017-09-08 17:13:00,350] worker_0 tf.server started.
[2017-09-08 17:13:00,354] making environment.
[2017-09-08 17:13:00,355] worker_0 is data_master: True
[2017-09-08 17:13:02,468] worker_0:envronment ok.
[2017-09-08 17:13:02,471] A3C_0: init() started
[2017-09-08 17:13:04,201] A3C_0: train op defined
[2017-09-08 17:13:04,269] A3C_0: init() done
[2017-09-08 17:13:04,271] worker_0:trainer ok.
[2017-09-08 17:13:05,215] worker_0: connecting to the parameter server... 


INFO:tensorflow:Restoring parameters from ./tmp/a3c_test/train/model.ckpt-0


[2017-09-08 17:13:05,260] Restoring parameters from ./tmp/a3c_test/train/model.ckpt-0
[2017-09-08 17:13:05,368] Press `Ctrl-C` to stop training and close launcher.
[2017-09-08 17:13:05,360] worker_1 tf.server started.
[2017-09-08 17:13:05,366] worker_2 tf.server started.
[2017-09-08 17:13:05,372] worker_3 tf.server started.
[2017-09-08 17:13:05,375] worker_4 tf.server started.
[2017-09-08 17:13:05,378] worker_5 tf.server started.
[2017-09-08 17:13:05,382] making environment.
[2017-09-08 17:13:05,383] making environment.
[2017-09-08 17:13:05,384] worker_6 tf.server started.
[2017-09-08 17:13:05,391] making environment.
[2017-09-08 17:13:05,388] making environment.
[2017-09-08 17:13:05,388] making environment.
[2017-09-08 17:13:05,388] making environment.
[2017-09-08 17:13:05,389] worker_1 is data_master: False
[2017-09-08 17:13:05,390] worker_2 is data_master: False
[2017-09-08 17:13:05,390] worker_7 tf.server started.
[2017-09-08 17:13:05,393] worker_6 is data_master: False
[2017-09-08

Press `Ctrl-C` to stop training and close launcher.


[2017-09-08 17:13:06,488] worker_1:envronment ok.
[2017-09-08 17:13:06,491] A3C_1: init() started
[2017-09-08 17:13:06,604] worker_7:envronment ok.
[2017-09-08 17:13:06,607] A3C_7: init() started
[2017-09-08 17:13:06,620] worker_2:envronment ok.
[2017-09-08 17:13:06,624] A3C_2: init() started
[2017-09-08 17:13:06,643] worker_6:envronment ok.
[2017-09-08 17:13:06,647] A3C_6: init() started
[2017-09-08 17:13:06,671] worker_3:envronment ok.
[2017-09-08 17:13:06,677] A3C_3: init() started
[2017-09-08 17:13:06,693] worker_4:envronment ok.
[2017-09-08 17:13:06,698] A3C_4: init() started
[2017-09-08 17:13:06,702] worker_5:envronment ok.
[2017-09-08 17:13:06,722] A3C_5: init() started
[2017-09-08 17:13:09,667] A3C_1: train op defined
[2017-09-08 17:13:09,811] A3C_1: init() done
[2017-09-08 17:13:09,819] worker_1:trainer ok.
[2017-09-08 17:13:09,923] A3C_2: train op defined
[2017-09-08 17:13:09,927] A3C_6: train op defined
[2017-09-08 17:13:09,933] A3C_7: train op defined
[2017-09-08 17:13:09,9

...

INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


[2017-09-08 22:08:11,112] Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


[2017-09-08 22:13:11,111] Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt
Process BTgymServer-201:1:
Process BTgymServer-198:1:
Process BTgymDataFeedServer-196:1:
Process BTgymServer-200:1:
Process DrawCerebro-196:2:3494:
Process BTgymServer-202:1:
Process BTgymServer-199:1:
Process BTgymServer-203:1:
[2017-09-08 22:17:11,166] worker_1 has joined.Process BTgymServer-197:1:
Traceback (most recent call last):
Traceback (most recent call last):

Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/server.py", line 413, in run
    episode = cerebro.run(stdstats=True, preload=False, oldbuysell=True)[0]
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/

  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 324, in _next
    super(Strategy, self)._next()
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:9621)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 585, in _notify
    for analyzer in itertools.chain(self.analyzers, self._slave_analyzers):
[2017-09-08 22:17:13,254] worker_7 has joined.
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/plot/plot.py", line 191, in plot
    self.plotind(None, ptop, subinds=self.dplotsover[ptop])
  File "zmq/backend/cython/socket.pyx", line 145, in zmq.backend.cython.socket._recv_copy (zmq/backend/cython/socket.c:2344)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 324, in _next
    super(Strategy, self)._next()
  File "/User

In [59]:
env_config.update({'port': 5050, 'data_port': 5049})
env = BTgymEnv(**env_config)

Process BTgymServer-176:
Process BTgymDataFeedServer-175:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/dataserver.py", line 100, in run
    service_input = socket.recv_pyobj()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/server.py", line 318, in run
    service_input = socket.recv_pyobj()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/zmq/sugar/socket.py", line 491, in recv_pyobj
    msg = self.recv(flags)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/zmq/sugar/socket.py", line 491, in recv_pyobj
    msg = self.recv(flags)
  File "zmq/backend/cython/socket.pyx", line

In [ ]:
o = env.reset()

In [ ]:
o

In [ ]:
o,r,d,i = env.step(0)
print(o)

In [ ]:
env.close()